In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### implementation of basic ae

In [11]:
class Evolutionary_Algorithm:
    __slots__ = [
        "mutation_rate",
        "crossover_rate",
        "number_of_generations",
        "population_size",
        "function_to_optimize",
        "population",
        "problem_dim",
    ]

    def __init__(
        self,
        mutation_rate=0.7,
        crossover_rate=0.7,
        number_of_generations=50,
        population_size=100,
    ):
        self.population_size = population_size
        self.number_of_generations = number_of_generations
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.function_to_optimize = None
        self.population = None
        self.problem_dim = None

    def _generate_population(self, n_genes):
        self.population = np.random.uniform(-10, 10, (self.population_size, n_genes))

    def _mutation(self, individual):
        for i in range(len(individual)):
            if np.random.rand() < self.mutation_rate:
                individual += np.random.normal(0, 1, self.problem_dim)
        return individual

    def _crossover(self, parent1, parent2):
        if np.random.rand() < self.crossover_rate:
            crossover_point = np.random.randint(1, self.problem_dim - 1)
            child1 = np.concatenate(
                (parent1[:crossover_point], parent2[crossover_point:])
            )
            child2 = np.concatenate(
                (parent2[:crossover_point], parent1[crossover_point:])
            )
            return child1, child2
        return parent1, parent2

    def _evaluate_population(self):
        fitness = np.zeros(len(self.population))
        for i, individual in enumerate(self.population):
            fitness[i] = self.function_to_optimize(individual)
        return fitness

    def _select_individual(self, fitness):
        # roulette wheel selection
        probabilities = fitness / np.sum(fitness)
        return self.population[
            np.random.choice(range(self.population_size), p=probabilities)
        ]

    def _visualize_individual(self, individual):
        print(individual)

    def optimize(self, function_to_optimize):
        self.function_to_optimize = function_to_optimize
        self.problem_dim = function_to_optimize.__code__.co_argcount
        self._generate_population(self.problem_dim)
        print(self.population)
        print(self.problem_dim)
        # for generation in range(self.number_of_generations):
        #     fitness = self._evaluate_population()
        #     new_population = np.zeros((self.population_size, self.problem_dim))
        #     for i in range(0, self.population_size, 2):
        #         parent1 = self._select_individual(fitness)
        #         parent2 = self._select_individual(fitness)
        #         child1, child2 = self._crossover(parent1, parent2)
        #         new_population[i] = self._mutation(child1)
        #         new_population[i + 1] = self._mutation(child2)
        #     self.population = new_population
        # best_individual = self.population[np.argmax(fitness)]
        # return best_individual

### definition of objective functions 

In [12]:
def rastrigin(*args):
    return 10 * len(args) + np.sum(args**2 - 10 * np.cos(2 * np.pi * args))


def rastrigin_5d(x, y, z, w, v):
    return rastrigin(x, y, z, w, v)


def quadratic_3d(x, y, z):
    return x**2 + y**2 + 2 * z**2

In [13]:
ea = Evolutionary_Algorithm()
ea.optimize(quadratic_3d)

[[ 9.07543178e+00  2.19704061e+00 -1.65684487e+00]
 [ 6.64572103e+00  1.98764783e+00 -8.74648055e+00]
 [ 2.61623592e+00 -3.44536972e+00  4.94083804e+00]
 [ 3.96885488e+00 -4.52855306e-01  9.08551336e+00]
 [ 4.87022465e+00 -9.75728706e+00  2.32141254e+00]
 [ 1.63523560e+00 -7.70795087e+00 -1.80493265e+00]
 [ 3.27972158e+00 -8.10072769e+00  1.36421670e+00]
 [ 9.88541230e+00 -8.87803045e+00  2.83035856e+00]
 [-3.52163749e+00 -5.29209372e+00 -6.62772913e+00]
 [ 9.11278418e+00 -1.01823678e+00 -3.88356004e-01]
 [-5.99065479e-01 -8.68608389e+00  5.83730391e+00]
 [-3.19238931e+00  8.53717483e+00  9.08745057e+00]
 [-6.43559235e-01 -1.26354136e-01 -7.15246333e+00]
 [-8.97415682e+00  4.86805107e+00  1.50857678e+00]
 [ 9.74006686e+00  1.73002782e+00 -9.82474174e+00]
 [-7.90534650e+00 -8.39877984e+00 -9.09168582e+00]
 [-8.34949864e+00 -8.38812683e+00  1.77554293e+00]
 [-9.45420969e-01  6.89737830e+00 -4.73712201e+00]
 [-4.94447016e+00  3.97161220e+00  9.34422997e+00]
 [ 7.54879100e-01  5.10640867e+